## Tello Drone Green Object Tracking with OpenCV

Imports and Setup

In [ ]:
pip install -r requirements.txt


In [ ]:
from djitellopy import Tello
import cv2
import numpy as np
import time
import os


Create Output Folder

In [ ]:
# Save videos in a dedicated folder
os.makedirs("output_videos", exist_ok=True)


Connect to Tello and Check Battery

In [ ]:
tello = Tello()
tello.connect()

battery = tello.get_battery()
print(f"Battery: {battery}%")

if battery < 20:
    print("Battery too low. Aborting.")
    exit()


Start Video Stream and Prepare Output

In [ ]:
tello.streamon()
frame_reader = tello.get_frame_read()

# Timestamped output filename
timestamp = time.strftime("%Y%m%d_%H%M%S")
video_filename = f"output_videos/tello_tracking_{timestamp}.avi"

# Configure video writer
fourcc = cv2.VideoWriter_fourcc(*'XVID')
video_out = cv2.VideoWriter(video_filename, fourcc, 20.0, (640, 480))


Auto Takeoff

In [ ]:
tello.takeoff()
time.sleep(2)


 Define Constants

In [ ]:
frame_center_x = 320
frame_center_y = 240
tolerance = 50
start_time = time.time()
max_duration = 180  # seconds
last_seen_time = time.time()


 Main Loop Track and Move

In [ ]:

try:
    while True:
        frame = frame_reader.frame
        if frame is None:
            print("Frame not received, skipping...")
            continue

        frame = cv2.resize(frame, (640, 480))
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

        # Green detection range
        lower_green = np.array([30, 40, 40])
        upper_green = np.array([90, 255, 255])
        mask = cv2.inRange(hsv, lower_green, upper_green)
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        valid_contour_found = False
        for cnt in contours:
            area = cv2.contourArea(cnt)
            if area > 1000:
                valid_contour_found = True
                x, y, w, h = cv2.boundingRect(cnt)
                cx = x + w // 2
                cy = y + h // 2
                last_seen_time = time.time()

                # Draw bounding box and center point
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                cv2.circle(frame, (cx, cy), 5, (0, 255, 0), -1)
                cv2.putText(frame, f"({cx}, {cy}) Area: {int(area)}", (x, y - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

                print(f" Green object at: ({cx}, {cy}) | Area: {area:.1f}")

                # Motion commands
                if cx < frame_center_x - tolerance:
                    print("→ Move LEFT")
                    tello.move_left(20)
                elif cx > frame_center_x + tolerance:
                    print("→ Move RIGHT")
                    tello.move_right(20)

                if cy < frame_center_y - tolerance:

                    print("→ Move UP")
                    tello.move_up(20)
                elif cy > frame_center_y + tolerance:
                    print("→ Move DOWN")
                    tello.move_down(20)

                if area < 3000:
                    print("→ Move FORWARD")
                    tello.move_forward(20)
                elif area > 25000:
                    print("→ Move BACKWARD")
                    tello.move_back(20)
                else:
                    print("→ HOLD POSITION")

                break

        if not valid_contour_found and time.time() - last_seen_time > 5:
            print(" No green detected, rotating...")
            tello.rotate_clockwise(30)
            time.sleep(1)
            last_seen_time = time.time()

        # Overlay timestamp and center dot
        elapsed = int(time.time() - start_time)
        cv2.putText(frame, f"Time: {elapsed}s", (10, 470),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
        cv2.circle(frame, (frame_center_x, frame_center_y), 5, (0, 0, 255), -1)

        # Write and show
        video_out.write(frame)
        print(f"Frame at {elapsed}s written to video.")

        cv2.imshow("Tello Camera", frame)
        cv2.imshow("Green Mask", mask)

        # Quit on ESC
        if cv2.waitKey(1) & 0xFF == 27:
            print(" ESC pressed — landing...")
            break

        # Timeout
        if time.time() - start_time > max_duration:
            print(" Max duration reached — landing.")
            break

        time.sleep(0.05)

except KeyboardInterrupt:
    print("KeyboardInterrupt — landing.")

finally:
    tello.land()
    tello.streamoff()
    cv2.destroyAllWindows()
    video_out.release()
    print(f" Drone landed. Video saved to {video_filename}")
    print(f" Total duration recorded: {int(time.time() - start_time)} seconds")
